In [1]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, jaccard_score, f1_score, precision_score, recall_score
from skimage.metrics import structural_similarity as ssim
from sklearn import preprocessing
from keras.utils import to_categorical
from keras import regularizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Dropout, Lambda, GlobalAveragePooling2D
from keras.src.legacy.preprocessing.image import ImageDataGenerator

import random
import re
import glob
from tqdm import tqdm
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
import gc

2025-05-10 13:56:40.211478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-10 13:56:40.277783: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-10 13:56:40.358766: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-10 13:56:40.975244: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs disponíveis: {gpus}")
else:
    print("Nenhuma GPU encontrada.")

GPUs disponíveis: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1746896217.838775   18819 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746896219.820211   18819 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746896219.820306   18819 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# nome = 'PACS'
nome = 'Yildirim'
#nome = 'Todos'

imgs_path = 'Bases/Dataset' + nome

output_dir = "Results/testes-CNN-MobileNet (0.1%)/"
os.makedirs(output_dir, exist_ok=True)

TEST_SIZE = 0.1
VALIDATION_SIZE = 0.1

EPOCHS = 200
BATCH_SIZE = 32

HIGH_FREQ_THRESHOLD = 0.1
NUM_FOLD = 5

In [4]:
def get_next_filename(output_folder, base_name, type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    image_index = 0
    
    while True:
        output_filename = f"{base_name}_{image_index}.{type}"
        output_path = os.path.join(output_folder, output_filename)
        
        if not os.path.exists(output_path):
            return output_filename
            
        image_index += 1

In [5]:
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(0)) if match else 0

In [6]:
def sort_files_numerically(file_paths):
    return sorted(file_paths, key=lambda x: extract_number(x))

In [7]:
def read_dataset(path, jpg, png):
    print(f'Reading dataset...\n')
    
    img_type = []
    images = []
    image_paths = []

    if jpg:
        img_type.append('*.jpg')
    if png:
        img_type.append('*.png')

# Normal

    normal_path = os.path.join(path, 'Train', 'Normal')
    print(f'Reading Normal images from: {normal_path}')
    
    for img_type_pattern in img_type:
        img_paths = glob.glob(os.path.join(normal_path, img_type_pattern))
        img_paths = sort_files_numerically(img_paths)
        
        for img_path in img_paths:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            image_paths.append(img_path)

# Kidney

    kidney_stone_path = os.path.join(path, 'Train', 'Kidney_stone')
    print(f'Reading Kidney_stone images from: {kidney_stone_path}')
    
    for img_type_pattern in img_type:
        img_paths = glob.glob(os.path.join(kidney_stone_path, img_type_pattern))
        img_paths = sort_files_numerically(img_paths)
        
        for img_path in img_paths:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            image_paths.append(img_path)

# ...
    
    images = np.array(images)
    image_paths = np.array(image_paths)

    return images, image_paths

In [8]:
all_X, all_image_paths = read_dataset(path=imgs_path, jpg=True, png=True)

Reading dataset...

Reading Normal images from: Bases/DatasetYildirim/Train/Normal
Reading Kidney_stone images from: Bases/DatasetYildirim/Train/Kidney_stone


In [9]:
output_dir_shift = "All_fshift"
os.makedirs(output_dir_shift, exist_ok=True)

output_dir_spectrum = "All_freqs"
os.makedirs(output_dir_spectrum, exist_ok=True)

def freq_spec(image, threshold, add_noise):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)

#   global image_counter
#
#   magnitude_temp = 20 * np.log(np.abs(fshift) + 1)
#
#   np.save(os.path.join(output_dir_shift, f"fshift_slice_{image_counter}.npy"), fshift)
#   
#   spectrum_path = os.path.join(output_dir_spectrum, f"spectrum_slice_{image_counter}.png")
#   cv2.imwrite(spectrum_path, magnitude_temp)
#
#   image_counter += 1

    if add_noise:
        amplification_factor = 0.5
        rows, cols = image.shape
        noise_size = int(np.sqrt(threshold * rows * cols))

        # Selecionar aleatoriamente um dos quatro cantos
        corner = np.random.randint(0, 4)

        # Aplicar ruído no canto selecionado
        if corner == 0:  # Canto superior esquerdo
            fshift[:noise_size, :noise_size] = 1
        elif corner == 1:  # Canto superior direito
            fshift[:noise_size, cols - noise_size:] = 1
        elif corner == 2:  # Canto inferior esquerdo
            fshift[rows - noise_size:, :noise_size] = 1
        else:  # Canto inferior direito
            fshift[rows - noise_size:, cols - noise_size:] = 1

    magnitude_spectrum_high = 20 * np.log(np.abs(fshift) + 1)

    return magnitude_spectrum_high

In [10]:
all_specs = []
all_labels = []

for i, img in enumerate(all_X):
    all_specs.append(freq_spec(img, HIGH_FREQ_THRESHOLD/100.0, add_noise=False))
    all_labels.append(0)

for i, img in enumerate(all_X):
    all_specs.append(freq_spec(img, HIGH_FREQ_THRESHOLD/100.0, add_noise=True))
    all_labels.append(1)

all_image_paths = np.concatenate([all_image_paths, all_image_paths])

all_image_paths = np.array(all_image_paths)
all_specs = np.array(all_specs)
all_labels = np.array(all_labels)

In [11]:
print("Quantidade das imagens:", all_specs.shape)
print("Quantidade dos paths de imagens:", all_image_paths.shape)
print("Exemplo dos labels (False = original, True = com ruído):", all_labels)

Quantidade das imagens: (3598, 224, 224)
Quantidade dos paths de imagens: (3598,)
Exemplo dos labels (False = original, True = com ruído): [0 0 0 ... 1 1 1]


In [12]:
X_train, X_test, y_train, y_test, paths_train, paths_test = train_test_split(
        all_specs, all_labels, all_image_paths,
        test_size=TEST_SIZE, 
        stratify=all_labels,
#       random_state=53
    )

In [13]:
def plot_specs(specs, title):
    total_images = len(specs)
    imgs_per_figure = 100
    cols = 5
    rows = (imgs_per_figure + cols - 1) // cols

    for start in range(0, total_images, imgs_per_figure):
        plt.figure(figsize=(cols * 3, rows * 3))
        end = min(start + imgs_per_figure, total_images)
        
        for i in range(start, end):
            plt.subplot(rows, cols, i - start + 1)
            plt.imshow(specs[i], cmap='gray')
            plt.title(f"{title} - freq spec: img {i+1}")
            plt.axis('off')
            
        plt.tight_layout()
        plt.show()

In [14]:
# Plotar todos os espectros de teste
#plot_specs(X_test, title="test")

In [15]:
#pausa de teste

In [16]:
# Define as layers do modelo MobileNet
def model_mobilenet():
# Camada de entrada para imagens em escala de cinza
    inputs = Input(shape=(224, 224, 1))
    
# 1. Conversão para 3 canais (replicando o canal único)
    x = Lambda(
        lambda x: tf.stack([x[..., 0]]*3, axis=-1),
        output_shape=(224, 224, 3)
    )(inputs)
    
# 2. Pré-processamento específico do MobileNet
    x = tf.keras.applications.mobilenet.preprocess_input(x)
    
# 3. Carregar MobileNet com pesos pré-treinados
    base_model = tf.keras.applications.MobileNet(
        weights='imagenet',
        include_top=False,
        input_tensor=x
    )
    
# 4. Adicionar novas camadas no topo
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(150, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)
    
    x = Dense(100, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)
    
    outputs = Dense(2, activation='softmax')(x)
    
# Compilar modelo
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='Adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
acc = []
jacc = []
f1 = []
prec = []
rec = []

# Configurar K-Fold com random_state fixo para reprodutibilidade
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=53)
kfold = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=53)

# Verificar quais folds já foram completados
completed_folds = []
for f in range(1, NUM_FOLD + 1):
    model_path = output_dir + f'model_{nome}_fold_{f}.keras'
    if os.path.exists(model_path):
        completed_folds.append(f)
print(f"Folds concluídos: {completed_folds}")

fold_no = 1
histories = []
metrics = []

for train_idx, val_idx in kfold.split(X_train, y_train):
# Pular folds já concluídos
    if fold_no in completed_folds:
        print(f"\nPulando fold {fold_no} (já concluído)")
        fold_no += 1
        continue

    print(f'\nTreinando Fold {fold_no}/{NUM_FOLD}')
    
# Split dos dados
    X_train_fold = X_train[train_idx]
    y_train_fold = y_train[train_idx]
    X_val_fold = X_train[val_idx]
    y_val_fold = y_train[val_idx]

# Pré-processamento final
    X_train_fold = np.expand_dims(X_train_fold, axis=-1)
    X_val_fold = np.expand_dims(X_val_fold, axis=-1)
    y_train_fold_cat = to_categorical(y_train_fold, 2)
    y_val_fold_cat = to_categorical(y_val_fold, 2)

# Criar novo modelo para cada fold
    model = model_mobilenet()

# Checkpoint com nome do fold
    checkpoint_filepath = output_dir + f'model_{nome}_fold_{fold_no}.keras'
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=False,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

# Calcula tempo (start)
    start_time = time.time()
    
# Treinar modelo
    history = model.fit(
        X_train_fold, y_train_fold_cat,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val_fold, y_val_fold_cat),
        callbacks=callbacks,
        verbose=1
    )

# Calcula tempo (end)
    end_time = time.time()

    training_time = end_time - start_time
    print(f"\nO modelo demorou {training_time:.2f} segundos para treinar.")

# Coletar métricas e salvar modelo
    predictions = model.predict(X_val_fold)
    y_pred = np.argmax(predictions, axis=1)
    
    metrics.append({
        'fold': fold_no,
        'report': classification_report(y_val_fold, y_pred, output_dict=True, zero_division=0),
        'matrix': confusion_matrix(y_val_fold, y_pred)
    })

# Métricas de classificação (por fold)
    acc.append(accuracy_score(y_val_fold, y_pred))
    jacc.append(jaccard_score(y_val_fold, y_pred))
    f1.append(f1_score(y_val_fold, y_pred))
    prec.append(precision_score(y_val_fold, y_pred))
    rec.append(recall_score(y_val_fold, y_pred))

# Salvar métricas em um arquivo .txt
    metrics_filename = os.path.join(output_dir, f'metrics_{nome}_fold_{fold_no}.txt')
    with open(metrics_filename, 'w') as f:
        f.write(f"Fold {fold_no} Metrics:\n")
        f.write(f"Accuracy: {acc[-1]}\n")
        f.write(f"Jaccard Score: {jacc[-1]}\n")
        f.write(f"F1 Score: {f1[-1]}\n")
        f.write(f"Precision: {prec[-1]}\n")
        f.write(f"Recall: {rec[-1]}\n")
        f.write("\nClassification Report:\n")
        f.write(classification_report(y_val_fold, y_pred, zero_division=0))
        f.write("\nConfusion Matrix:\n")
        f.write(np.array2string(confusion_matrix(y_val_fold, y_pred)))

# Limpeza de memória
    del model
    tf.keras.backend.clear_session()
    gc.collect()

# Salvar checkpoint a cada 3 folds
    if fold_no % 3 == 0:
        print(f"\nCheckpoint: Folds {fold_no-2}-{fold_no} concluídos")

    fold_no += 1

Folds concluídos: []

Treinando Fold 1/5


/tmp/ipykernel_18819/366968437.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNet(
I0000 00:00:1746896373.176902   18819 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746896373.177324   18819 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746896373.177431   18819 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746896373.799102   18819 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel m

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


2025-05-10 13:59:46.889320: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 519823360 exceeds 10% of free system memory.
2025-05-10 13:59:47.438251: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 519823360 exceeds 10% of free system memory.


Epoch 1/200


...

/tmp/ipykernel_18819/366968437.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNet(
2025-05-10 14:06:15.342345: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 519823360 exceeds 10% of free system memory.
2025-05-10 14:06:15.762170: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 519823360 exceeds 10% of free system memory.


Epoch 1/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.5262 - loss: 3.9899
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to Results/testes-CNN-MobileNet (0.1%)/model_Yildirim_fold_2.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 33s 222ms/step - accuracy: 0.5266 - loss: 3.9816 - val_accuracy: 0.5000 - val_loss: 2.5732
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9262 - loss: 1.5124
Epoch 2: val_accuracy improved from 0.50000 to 0.91204, saving model to Results/testes-CNN-MobileNet (0.1%)/model_Yildirim_fold_2.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.9267 - loss: 1.5083 - val_accuracy: 0.9120 - val_loss: 0.9710
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9957 - loss: 0.6033
Epoch 3: val_accuracy improved from 0.91204 to 1.00000, saving model to Results/testes-CNN-MobileNet (0.1%)/model_Yildirim_fold_2.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.9958 - loss: 0.6021 - val_accuracy: 1.0000 - val

/tmp/ipykernel_18819/366968437.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNet(
2025-05-10 14:11:58.420280: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 519823360 exceeds 10% of free system memory.


Epoch 1/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.4929 - loss: 3.9888
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to Results/testes-CNN-MobileNet (0.1%)/model_Yildirim_fold_3.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 30s 174ms/step - accuracy: 0.4936 - loss: 3.9808 - val_accuracy: 0.5000 - val_loss: 4.1932
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.8219 - loss: 1.6461 
Epoch 2: val_accuracy did not improve from 0.50000
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 103ms/step - accuracy: 0.8232 - loss: 1.6410 - val_accuracy: 0.5000 - val_loss: 8.0354
Epoch 3/200
80/81 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 0.5612
Epoch 3: val_accuracy improved from 0.50000 to 0.95062, saving model to Results/testes-CNN-MobileNet (0.1%)/model_Yildirim_fold_3.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 1.0000 - loss: 0.5589 - val_accuracy: 0.9506 - val_loss: 0.4672
Epoch 4/200
80/81 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.000

/tmp/ipykernel_18819/366968437.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNet(


In [ ]:
print("Accuracy: "+ str(np.mean(acc)) + "+- " + str(np.std(acc)))
print("Jaccard: "+ str(np.mean(jacc)) + "+- " + str(np.std(jacc)))
print("Dice: "+ str(np.mean(f1)) + "+- " + str(np.std(f1)))
print("Precision: "+ str(np.mean(prec)) + "+- " + str(np.std(prec)))
print("Recall: "+ str(np.mean(rec)) + "+- " + str(np.std(rec)))